In [1]:
%%capture
!pip install bertopic datasets transformers scipy evaluate

In [2]:
#Basics imports
from random import seed, choices
import pandas as pd
import numpy as np

#Evaluate the dataset
from bertopic import BERTopic
from datasets import load_dataset_builder, load_dataset, Dataset

#Evaluate the model
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import evaluate
import torch

#Fleiss Kappa import
from statsmodels.stats import inter_rater as irr

## Import Data

We decided to choose the offensive one because offensive is a notion more generalized and also ore tricky to detect. We think that this may be better for our case to study it.

In [ ]:
df = load_dataset("tweet_eval", "offensive")
df_builder = load_dataset_builder("tweet_eval", "offensive")

## Evaluate the Dataset
### Description

In [4]:
#Get the main description of the dataset
df_builder.info.description

'TweetEval consists of seven heterogenous tasks in Twitter, all framed as multi-class tweet classification. All tasks have been unified into the same benchmark, with each dataset presented in the same format and with fixed training, validation and test splits.\n'

In [5]:
#Get the features and so how the dataset is composed
df_builder.info.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['non-offensive', 'offensive'], id=None)}

In [6]:
#Get infos about splits
df_builder.info.splits

{'train': SplitInfo(name='train', num_bytes=1648061, num_examples=11916, shard_lengths=None, dataset_name='tweet_eval'),
 'test': SplitInfo(name='test', num_bytes=135473, num_examples=860, shard_lengths=None, dataset_name='tweet_eval'),
 'validation': SplitInfo(name='validation', num_bytes=192417, num_examples=1324, shard_lengths=None, dataset_name='tweet_eval')}

We decided to code a function that prints the ratio of each class for train, test and validation split.

In [7]:
import datasets
def get_ratio(df: datasets.dataset_dict.DatasetDict) -> None:
    """
    Get the ratio of each class for the split part given as parameter
    Args:
        df: Tweet_eval/offensive dataset.
    """
  
    for split in df:
      classes = df[split]['label']
      lg = len(classes)
      non_offensive = classes.count(0)
      offensive = classes.count(1)
      ratio_neg = round(non_offensive / lg * 100, 2)
      ratio_pos = round(offensive / lg * 100, 2)

      print("Ratio for the", split, "dataset")
      print("Non offensive values :", non_offensive)
      print("Offensive values :", offensive)
      print("We have", ratio_neg, "% of non_offensive and", ratio_pos, "% of offensive")
      print("----------")

In [8]:
get_ratio(df)

Ratio for the train dataset
Non offensive values : 7975
Offensive values : 3941
We have 66.93 % of non_offensive and 33.07 % of offensive
----------
Ratio for the test dataset
Non offensive values : 620
Offensive values : 240
We have 72.09 % of non_offensive and 27.91 % of offensive
----------
Ratio for the validation dataset
Non offensive values : 865
Offensive values : 459
We have 65.33 % of non_offensive and 34.67 % of offensive
----------


Classes are very imbalanced and this may cause some troubles for our models after.

### Examples

In [9]:
df['test']['text'][:20]

['#ibelieveblaseyford is liar she is fat ugly libreal #snowflake she sold her  herself to get some cash !! From dems and Iran  ! Why she spoke after  #JohnKerryIranMeeting ?',
 '@user @user @user I got in a pretty deep debate with my friend and she told me that latinos for Trump and blacks for Trump were paid supporters 😂 then I said you mean antifa are paid domestic terrorist and she said No they are  anti-fascist then I said they are the fascist are you kidding me?!',
 '...if you want more shootings and more death, then listen to the ACLU, Black Lives Matter, or Antifa. If you want public safety, then listen to the police professionals who have been studying this for 35 years."" -AG Jeff Sessions',
 'Angels now have 6 runs. Five of them have come courtesy Mike Trout homers. Trout connects on a 3-2 pitch with runners on second and third',
 '#Travel #Movies and Unix #Fortune combined  Visit #Salisbury, see the sights, but wherever you go there you are, the same old you, why not travel 

### Topics Model
#### Creation of the topics

In [10]:
#Create a BERT Topic model using the all-MiniLM-L6-v2 model pre-trained model from the sentence-transformers librairy
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2")

In [ ]:
#Retrieve same values each time
seed(23)

topics, probs = topic_model.fit_transform(df['train']['text'])

In [12]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,5211,-1_is_he_to_you
1,0,1011,0_gun_control_guns_laws
2,1,413,1_liberals_they_liberal_their
3,2,211,2_kavanaugh_judge_maga_accuser
4,3,192,3_conservatives_conservative_they_are
...,...,...,...
123,122,11,122_ford_dr_claims_letter
124,123,11,123_quran_books_bible_hindus
125,124,11,124_worst_terrible_cage_worse
126,125,10,125_killed_antifa_zhe_xhe


#### Visualization

In [13]:
topic_model.visualize_topics()

In [14]:
topic_model.visualize_barchart()

## Evaluate a model
### Results on main metrics

In [15]:
MODEL = f"cardiffnlp/twitter-roberta-base-offensive"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [28]:
type(model)

transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification

In [ ]:
evaluator = evaluate.evaluator("text-classification")
metrics = evaluate.combine(['accuracy', "recall", "precision", "f1"])


results = evaluator.compute(
    model_or_pipeline=model,
    data=df['test'],
    metric=metrics,
    label_mapping={"LABEL_0": 0.0, "LABEL_1": 1.0},
    tokenizer=tokenizer
    )

In [18]:
results

{'accuracy': 0.8593023255813953,
 'recall': 0.6666666666666666,
 'precision': 0.7960199004975125,
 'f1': 0.7256235827664399,
 'total_time_in_seconds': 9.022430702000008,
 'samples_per_second': 95.3179944966896,
 'latency_in_seconds': 0.010491198490697684}

### Extract tweets

In [ ]:
!wget https://raw.githubusercontent.com/mvonwyl/epita/master/NLP/2022/08/tweets.json

In [20]:
tweets_pandas = pd.read_json('tweets.json')
tweets = list(tweets_pandas['text'])

In [29]:
import transformers
def get_preds(tweets: list, 
              model: transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification
              ) -> np.ndarray:
  """
    Get predictions from the tweets.json file with the the model pretrained.
    Args:
        - tweets (list): the tweets
        - model : our pretrained model
    Returns:
        - preds : our predicitons
  """

  #Use a loader to batch our tweets
  loader = torch.utils.data.DataLoader(tweets, batch_size=256)
  preds = []

  with torch.no_grad():
      for batch in iter(loader):
        #Tokenize each batch
        tokenized_batch = tokenizer(batch, truncation=True, padding=True, return_tensors="pt").to(model.device)
        
        #Get for each batch the predictions
        logits = model(**tokenized_batch).logits
        current_preds = torch.softmax(logits, dim=1)
        
        #Add them to the global preds
        preds.append(current_preds)

  return torch.cat(preds, dim=0).cpu().numpy()

In [ ]:
preds = get_preds(tweets, model)

In [86]:
preds[10]

array([0.6560972 , 0.34390283], dtype=float32)

In [70]:
a = [2,15,16]
np.flip(a)

array([16, 15,  2])

### Extract tweets
Here, we will extract following the subject :
- top 50 tweets where the model is the most confident
- top 50 in neutral class
- top 50 where the model is the less confident

In [179]:
#Global variables to define bornes for uncertained tweets
SUP = 0.51
INF = 0.49

def get_value(index: int, preds: np.ndarray, label: int, confident: bool) -> float:
  """
    Get the value from the preds
    Args :
          - index (int): index in preds
          - preds (np.ndarray): predictions from the model pretrained
          - label (int):
                        - 0 : non offensive
                        - 1 : offensive
          - confident (bool): boolean value if we want results where the model is confident or not
  Returns:
          - value (float) : the value found. If no one is found, we return -1
  """
  value = -1
  if confident:
    value = preds[index][label]

  #We're looking for uncertain values from the preds
  #We define bornes to obtain these specific values (~0.50)
  else:
    if preds[index][label] < SUP and preds[index][label] > INF:
      value = preds[index][label]

  return value


def get_top_tweets(preds: np.ndarray, tweets: list, label: int, confident=True, K=50) -> list:
  """
    Get a top 50 of tweets following what we want, from which class and if the model is confident or not
    Args :
        - preds (np.ndarray): predicitons
        - tweets (list): list of tweets
        - label (int):
                      - 0 : non offensive
                      - 1 : offensive
        - confident (bool): boolean value if we want results where the model is confident or not
        - K (int): Number of tweets we want
    Returns:
        - top_50 (list): list of 50 tweets
  """
  best_values = [] #best values in the preds (best min or best max following what we want)
  top_50 = [] #List of tweets

  for i in range(len(preds)):
    #get the value
    value = get_value(i, preds, label, confident)

    #For the case if confident = False
    if value == -1:
      continue

    #Fill the list of best values
    if len(best_values) < K:
      best_values.append(value)
    else:
      #Replace the minimal value in the list by an higher one
      if min(best_values) < value and len(best_values) == K:
        best_values[np.argmin(best_values)] = value
      
  #Retrieve the tweets by getting index related to the best values
  for value in best_values:
    indexes = np.where(preds == value)
    top_50.append(tweets[indexes[0][0]])
  
  return top_50

In [169]:
confident_tweets_offensive = get_top_tweets(preds, tweets, label=1)
confident_tweets_offensive[:5]

['Wah fuck out u pussy',
 'I wonder how long it took for him to write all that shit on his face',
 'Fauci, the repugnant little troll is hiding. He knows he is responsible for a lot if this covid shit, and he should be in prison for life!',
 'I think that’s commonly known as RETALIATION. Another mail in the coffin of American democracy. How do these fuckers get away with this crap',
 'Shut the hell up nobody give a shit about your bogus ass page stop scamming and cheating stats making rebounds 1 less on some stupid shit that wasn’t a fcking steal by ayton. I’m reporting this account']

In [146]:
confident_tweets_neutral = get_top_tweets(preds, tweets, label=0)
confident_tweets_neutral[:5]

["you're welcome, thank you for being patient and kind to us by the way &lt;33",
 "You are welcome, let's keep in touch😊",
 'thank you Shay🥺❤️',
 'Thanks so much LEU 😍',
 'i honestly love you so much, with all my heart &lt;3\nthanks for always putting a smile on my face!! love you ever so much!! @user']

In [150]:
uncertain_tweets_offensive = get_top_tweets(preds, tweets, label=1, confident=False)
uncertain_tweets_offensive[:5]

['Even if they didn’t exploit people to acquire their riches, how are you gonna be okay literally wasting thousands and thousands of dollars while there are still people who are homeless? While there are people skipping life saving medical treatments bc of the cost?',
 'Guys chill we got better things to do than touch grass',
 'giving a man who luvs to smoke a bouquet of weed + backwoods sounds fire',
 'Heavy on saying make sure my chicken fried hard 😭',
 "It's burned into my brain since childhood. Lol"]

We think that the model is doing a great job because when we want results where is very confident, none of them is wrongly classified. But, we need to find how uncertain tweets it found.


In [173]:
#Here it's the length with SUP = 0.51 and INF = 0.49
all_uncertain = get_top_tweets(preds, tweets, label=1, confident=False, K=10000)
len(all_uncertain)

74

In [178]:
#Here it's the length with SUP = 0.6 and INF = 0.4
all_uncertain_wide = get_top_tweets(preds, tweets, label=1, confident=False, K=10000)
len(all_uncertain_wide)

819

With these results, over 10K tweets, we think that the model is doing a great job.

### Wrongly classified

In [188]:
all_uncertain_wide[6]

'the day my lungs gonna collapse 😭😭'

We think that the model said that this is offensive due to the world "collapse". This is why it hesitate a lot.

In [191]:
all_uncertain_wide[19]

'i need my younger brother to leave. HE EATS EVERYTHING nobody needs a dozen eggs a day.'

Here, maybe the world "leave" influence the model in the wrong way.

## Annotate the data
We take 20 tweets where the model is uncertain. This is perfect for us because it's this specific case where we will see if we are agree when we will annotate these tweets.

In [157]:
uncertain_offensive_tweets = get_top_tweets(preds, tweets, label=1, confident=False, K=20)
cool_tweets = get_top_tweets(preds, tweets, label=0, confident=True, K=80)

extracted_tweets = uncertain_offensive_tweets + cool_tweets

In [158]:
extracted_tweets

["Not triggered point is bullying is obviously wrong. But now there's going to be people or certain people jumping on this bandwagon  sucks the fun and the debate aspect outta Twitter imo so we all have to watch our Ps &amp; Qs just craic killer now. Again bullying is definitely wrong.",
 'Guys chill we got better things to do than touch grass',
 'giving a man who luvs to smoke a bouquet of weed + backwoods sounds fire',
 'like damn they possibly made her say it so we wouldn’t get mad cuz we love her and we wouldn’t get mad at the company ://',
 'who has tickets for hoes on tour 🤞🏼🤞🏼❤️🔥🔥😂🥶🥶🥵',
 'http y’all go mess wit my music channel💯',
 'They can’t be disgraced or shamed because they have no grace or dignity.',
 'Heavy on saying make sure my chicken fried hard 😭',
 'Cyrus Bean is the most manipulative character tv has ever seen #Scandal',
 'On three way getting cussed out 😭😭😭😭😭',
 'This guy needs to quit wasting my time and buy my sprinter! 😡',
 'Web App is even worse',
 'i hope they

### Fleiss Kappa
We are 4 in this group, we must use the Fleiss Kappa score. To do so we recup from our excel file all the rating from each person. After that, we use the function fleiss_kappa from the statsmodels librairy.

Here only the first 20 tweets are uncertained one from the model. All the others are non offensive.

In [159]:
eliot = [0,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,1,1,0,1] + [0]*80
alex = [0,0,0,0,0,1,1,0,0,1,1,1,1,1,1,1,1,1,1,0] + [0]*80
tom = [0,0,0,1,0,0,1,0,0,1,1,1,1,1,0,1,1,1,0,0] + [0]*80
aurelien = [0,0,0,0,0,0,1,0,0,1,1,1,1,0,1,1,1,1,1,0] + [0]*80

annotators = [alex, eliot, tom, aurelien]

In [164]:
NB_TWEETS = 100

def create_table(annotators: list) -> list:
  """
    Create a table in 2D for the Fleiss Kappa score.
    Args:
        - annotators (list) : the list containing the annotations for each person of the group.
    Returns:
        - table_rating (list) : list of list of the ratings.
  """
  table_rating = []

  for i in range(NB_TWEETS):
    scores = [0,0]
    
    for annotator in annotators:
      # We decide here that the first column is for non_offensive
      # in the excel file, 0 = non offensive and 1 = offensive
      if annotator[i] == 0:
        scores[0] += 1
      
      else:
        scores[1] +=1
      
      table_rating.append(scores)

  return table_rating


In [165]:
table_rating = create_table(annotators)
kappa_score = irr.fleiss_kappa(table_rating, method='fleiss')

print("Fleiss Kappa score :", kappa_score)

Fleiss Kappa score : 0.7871774408087256


Following the wikipedia page, at the section **Interpretation**, if we have 0.6 < score < 0.8, it's considered as "*Substantial agreement*".